In [1]:
import pandas as pd
import numpy as np

# Select state

In [2]:
from ipywidgets import Select  

In [3]:
states = pd.read_csv("state_abbrv.tsv",sep="\t").set_index('state').abbreviation.to_dict()
state_widget = Select(description='Select a state', options=states,rows=10) 
state_widget

Select(description='Select a state', options={'Alabama ': 'AL', 'Alaska ': 'AK', 'Arizona ': 'AZ', 'Arkansas '…

# Get the data

In [4]:
import requests

In [50]:
rest_url = 'https://services9.arcgis.com/6Hv9AANartyT7fJW/ArcGIS/rest/services/USCounties_cases_V1/FeatureServer/0/query?where=ST_Abbr%3D%27{}%27&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=Countyname%2CST_Name%2CConfirmedb%2CConfirmed%2CPOP_ESTIMA&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=none&maxAllowableOffset=&geometryPrecision=&outSR=3819&datumTransformation=&applyVCSProjection=true&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token='.\
            format(state_widget.get_interact_value())

response = requests.get(rest_url)
covid_json = response.json()

# Process data

Get data into a format that can be used by Plotly.

In [51]:
geo_features = []
lat_longs = []
for rec in covid_json['features'] :
    new_rec = {}
    new_rec['properties'] = rec['attributes']
    new_rec['geometry'] = {'type' : 'Polygon' }
    new_rec['geometry']['coordinates'] = rec['geometry']['rings']
    lat_longs.append(new_rec['geometry']['coordinates'][0][0])
    geo_features.append(new_rec)

geos = { "type" : "FeatureCollection", "features" : geo_features}

Calculate center

In [53]:
center = pd.DataFrame(lat_longs,columns=['lon','lat']).mean().to_dict()

Convert to data frame

In [54]:
features = [ x['attributes'] for x in covid_json['features']]


df = pd.DataFrame(features)
df ['density'] = df.Confirmed / df.POP_ESTIMA * 1_000
df

,Countyname,ST_Name,Confirmedb,Confirmed,POP_ESTIMA,density
0,Adams,Wisconsin,14.743464,3,20348,0.147435
1,Ashland,Wisconsin,12.820513,2,15600,0.128205
2,Barron,Wisconsin,13.284917,6,45164,0.132849
3,Bayfield,Wisconsin,19.944156,3,15042,0.199442
4,Brown,Wisconsin,43.283797,114,263378,0.432838
...,...,...,...,...,...,...
67,Waukesha,Wisconsin,59.046523,238,403072,0.590465
68,Waupaca,Wisconsin,7.823502,4,51128,0.078235
69,Waushara,Wisconsin,8.243004,2,24263,0.082430
70,Winnebago,Wisconsin,18.126535,31,171020,0.181265


# Plot

In [31]:
import plotly.express as px
import plotly.graph_objects as go

In [55]:
fig = px.choropleth_mapbox(df,geojson=geos,
                           featureidkey='properties.Countyname',
                           locations='Countyname',
                           color='density',
                           mapbox_style="carto-positron", 
                           center=center, 
                           #opacity=0.5,
                           zoom=5,
                           hover_data=['Confirmed','POP_ESTIMA'],
                           #title='COVID cases per capita',
                           #height=650
                          )

In [56]:
go.FigureWidget(fig)

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'customdata': array([[     3,  20348],
  …